In [ ]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from typing import Annotated
from typing_extensions import TypedDict
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver

# ===== Stateクラスの定義 =====
class State(TypedDict):
    messages: Annotated[list, add_messages]

# ===== グラフの構築 =====
def build_graph(model_name):
    # グラフのインスタンスを作成
    graph_builder = StateGraph(State)

    # 言語モデルの定義
    llm = ChatOpenAI(model=model_name)

    # 検索ツールの定義
    tool = TavilySearchResults(max_results=2)
    tools = [tool]

    # 変更点：ツール定義の紐づけ
    llm_with_tools = llm.bind_tools(tools)

    # チャットボットノードの作成
    def chatbot(state: State):
        return {"messages": [llm_with_tools.invoke(state["messages"])]}
    
    # グラフにチャットボットノードを追加
    graph_builder.add_node("chatbot", chatbot)

    # ツールノードの作成
    tool_node = ToolNode(tools)

    # グラフにツールノードを追加
    graph_builder.add_node("tools", tool_node)

    # 条件付エッジの作成
    graph_builder.add_conditional_edges(
        "chatbot",
        tools_condition, # ツール呼出と判断したらツールノードを呼ぶ
    )

    # ツールが呼び出されるたびに、チャットボットに戻って次のステップを決定
    # ツールからチャットボットへの戻りエッジを作成
    graph_builder.add_edge("tools", "chatbot")

    # 開始ノードの指定
    graph_builder.set_entry_point("chatbot")

    # 記憶を持つ実行可能なステートグラフの作成
    memory = MemorySaver()
    return graph_builder.compile(checkpointer=memory)

# ===== グラフ実行関数 =====
def stream_graph_updates(graph: StateGraph, user_input: str):
    events = graph.stream(
        {"messages": [("user", user_input)]},
        {"configurable": {"thread_id": "1"}},
        stream_mode="values")
    # 結果をストリーミングで得る
    for event in events:
        print(event["messages"][-1].content, flush=True)

# ===== メイン実行ロジック =====
# 環境変数の読み込み
load_dotenv("../.env")
os.environ['OPENAI_API_KEY'] = os.environ['API_KEY']

# モデル名
MODEL_NAME = "gpt-4o-mini" 

# グラフの作成
# ソースコードを記述
graph = build_graph(MODEL_NAME)

# メインループ
# ソースコードを記述
while True:
    user_input = input("質問:")
    if user_input.strip()=="":
        print("ありがとうございました!")
        break
    stream_graph_updates(graph, user_input)

こんにちは！
こんにちは！どのようにお手伝いできますか？
1たす2は？
1たす2は3です。
台湾観光について検索結果を教えて

[{"url": "https://www.arukikata.co.jp/area/tw/", "content": "国と地域\n\n ヨーロッパ\n 北米\n 中南米\n オセアニア\n ハワイ\n リゾート\n アジア\n 中近東\n アフリカ\n 日本\n\n ウェブマガジン\n 特派員ブログ\n ガイドブック\n 筆者一覧\n 掲示板\n 運営会社\n お問い合わせ\n\nログイン / 会員登録\n\n# 台湾の基本情報と観光ガイド・有名観光スポット\n\n ### 台湾・台北の基本情報と観光ガイド・有名観光スポット\n ### 台湾・台中の基本情報と観光ガイド・有名観光スポット\n ### 台湾・台南の基本情報と観光ガイド・有名観光スポット\n ### 台湾・高雄の基本情報と観光ガイド・有名観光スポット\n\n ### 国立故宮博物院\n + 台北\n  + 観光\n\n  ### 龍山寺（台北）\n + 観光\n\n  ### 寶覺寺\n + 台中\n  + 観光\n\n  ### 国立自然科学博物館\n + 観光\n\n  ### 赤崁楼\n + 台南\n  + 観光\n\n  ### 祀典武廟\n\n  台湾関帝廟の総本山\n + 高雄\n  + 観光\n\n  ### 駁二藝術特區\n\n  アートが集まる倉庫群\n + 高雄\n  + 観光\n\n  ### 美麗島駅\n\n  美しいステンドグラスは必見\n\n## 台湾の観光ポイント\n\n### 食を楽しむ旅\n\n旅の目的が「食」という人もいるほど台湾グルメは大人気。夜市にはおいしい屋台があふれ、地元の人々と熱気あふれる夜のひとときを楽しむことができる。小籠包や牛肉麺、ルーロー飯といった台湾の名物料理はぜひ味わっておきたい。また、台南の伝統的な料理や、地方の特色ある食材を使った料理も、楽しみのひとつ。さらに、忘れてはならないのが台湾スイーツ。マンゴーかき氷やタピオカミルクティー、豆乳を固めたゼリーのような食感の豆花など、華やかでヘルシーなスイーツも存分に堪能したい。\n\n 台湾\n 飲食 [...] 台湾\n 飲食\n\n初めて&久しぶりの台湾旅でいま行くべ